In [1]:
# import kebutuhan library
import sqlite3
import re
import pandas as pd

In [2]:
# buka file csv dengan pandas
df = pd.read_csv('data.csv',encoding='ISO-8859-1')

# buka koneksi ke file database 'testdb.db'
conn = sqlite3.connect('testdb.db')


In [3]:
# import sheet "data" ke table data
df.to_sql(name="data",con=conn,if_exists='replace',index=False)
conn.commit()



In [4]:
# get data dari table ABUSIVE, hasil query di jadikan dataframe pandas
dfAbusive = pd.read_sql_query("SELECT * FROM ABUSIVE", conn)
conn.close()

# print(dfAbusive.head(100))

In [5]:
# function untuk menyensor kata-kata abusive
def remove_abusive_words(text, df = dfAbusive):
    
    # get list data abusive
    abusive_words = df['abusive'].tolist()
    
    # check apakah ada kata-kata abusive atau kasar dalam data frame abusive_words
    pattern = "|".join(re.escape(text) for text in abusive_words)
    
    # jika ada sensor dengan ####
    cencored_text = re.sub(pattern, "#######", text, flags=re.IGNORECASE)
    
    return cencored_text



In [6]:
# validasi apakah text mengandung uncode 
def is_contains_unicode(text):
    pattern = r'\\x[0-9a-fA-F]{2}.'

    return bool(re.search(pattern, text))

In [7]:
# function untuk menghapus escape chars seperti \\n atau \\t
def remove_escape_chars(text):
    pattern = r'\\[ntrbafv"\'\\]'
    clean_text =  re.sub(pattern, '', text)
    
    return clean_text

In [8]:
# function cleansing text
def cleanText(text):
    cleaned_text = text
    
    
    if is_contains_unicode(text):
        pattern = r'\\x[0-9a-fA-F]{2}|\\'
        clean_unicode = re.sub(pattern, '', text)
        remove_escape_chars(clean_unicode)
        cleaned_text = clean_unicode
    else:
        cleaned_text = remove_escape_chars(cleaned_text)
        
    
    
    return cleaned_text

In [ ]:
# untuk menjalankan flask dan swagger
from flask import Flask
from flask_restx import Api, Resource

app = Flask(__name__)
api = Api(app)

@api.route('/cleansing')
class CleansingData(Resource):
    def get(self):
        
        # connect ke database
        with sqlite3.connect('testdb.db') as conn:
            # object koneksi database
            cursor = conn.cursor()
            
            # jalanin query
            cursor.execute("SELECT Tweet FROM data LIMIT 100")
            
            #tampil hasil query
            rows = cursor.fetchall()
            
            #hasil query di cleansing
            text = [remove_abusive_words(cleanText(str(row[0]))) for row in rows]

            # print(remove_abusive_words(text))
            
            #hasi cleansing di tampilkan ke dalam json
            return {'data': text}
    

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)  # Set use_reloader to False to run in Jupyter


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Dec/2023 17:40:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 17:40:35] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Dec/2023 17:40:35] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Dec/2023 17:40:35] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Dec/2023 17:40:35] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Dec/2023 17:40:36] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 17:40:36] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 304 -
127.0.0.1 - - [01/Dec/2023 17:40:43] "GET /cleansing HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 19:18:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2023 19:18:30] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Dec/2023 19:18:30] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [01/Dec/2023 19:18:30] "GET /swaggerui/swagger-ui-bund